# Tugas Besar A
IF3270 Pembelajaran Mesin<br>
Forward Propagation - Feed Forward Neural Network (FFNN)

Developed by:
1. K01 13520010 - Ken Kalang Al Qalyubi
2. K01 13520036 - I Gede Arya Raditya Parameswara
3. K02 13520061 - Gibran Darmawan
4. K03 13520119 - Marchotridyo

## Library

In [ ]:
import pandas as pd
import os, subprocess, sys
import json, math, typing, copy
import numpy as np
import matplotlib as plt

## Enum

In [ ]:
class LayerEnum:
    INPUT = "INPUT"
    HIDDEN = "HIDDEN"
    OUTPUT = "OUTPUT"

class ActivationFuncEnum:
    SIGMOID = "SIGMOID"
    LINEAR = "LINEAR"
    RELU = "RELU"
    SOFTMAX = "SOFTMAX"

## File Utility

In [ ]:
class FileUtility:
    @staticmethod
    def import_json(file_name):
        with open(file_name) as json_file:
            return json.load(json_file)

    @staticmethod
    def export_json(file_name, data):
        with open(file_name, 'w') as outfile:
            json.dump(data, outfile)

## Neuron

In [ ]:
class Neuron:
    def __init__(self, layer: LayerEnum, activation: ActivationFuncEnum, index: int, weights: list, bias: float, value: float = 0.0):
        self.layer = layer
        self.activation = activation
        self.index = index
        self.weights = weights
        self.bias = bias
        self.value = 0.0

    def __str__(self):
        return "Neuron: " + str(self.index) + " " + str(self.layer) + " " + str(self.activation) + " " + str(self.value)

    def __repr__(self):
        return self.__str__()

    def set_value(self, value: float):
        self.value = value

    def get_value(self):
        return self.value

    def get_activation(self):
        return self.activation

    def get_layer(self):
        return self.layer

    def get_index(self):
        return self.index

    def get_weights(self):
        return self.weights

    def get_bias(self):
        return self.bias

## ANN Graph

In [ ]:
class ANNGraph:
    def __init__(self, file_config_path: str):
        self.file_path = file_config_path
        self.config = None
        self.layers = []

        self.build_ann_graph()

    def build_ann_graph(self):
        self.config = FileUtility.import_json(self.file_path)

        for layer in self.config["layers"]:
            for n